# **Understanding Word Embeddings**

## **One-hot Vectors**

In [ ]:
tokens = ["I", "love", "you", "3000"]

# Build vocabulary
vocab = {token: i for i, token in enumerate(tokens)}
vocab_size = len(vocab)

# One-hot encode
def one_hot(index, size):
    vec = [0] * size
    vec[index] = 1
    return vec

one_hot_vectors = [one_hot(vocab[token], vocab_size) for token in tokens]

print("Vocabulary:", vocab)
print("One-hot vectors:")
for t, v in zip(tokens, one_hot_vectors):
    print(t, "→", v)

Vocabulary: {'I': 0, 'love': 1, 'you': 2, '3000': 3}
One-hot vectors:
I → [1, 0, 0, 0]
love → [0, 1, 0, 0]
you → [0, 0, 1, 0]
3000 → [0, 0, 0, 1]


1. **PyTorch-based Custom Embeddings:**

Builds embeddings from scratch using neural networks.
- N-Gram Language Models: Predict the next word based on previous words
- Continuous Bag-of-Words (CBOW): Predict a target word from surrounding context words
- SkipContinuous Bag-of-Words (CBOW): Predict surrounding context words based on target/center word


2. **FastText**:

Extends traditional word embeddings by incorporating subword information. This approach:
- Handles out-of-vocabulary words by breaking them into character n-grams
- Understands morphological relationships (e.g., "drive," "driving," "driver")
- Works with both Skip-gram and CBOW architectures


3. **Word2Vec:**

Classic Word2Vec algorithm that revolutionized NLP. You'll implement:
- Skip-gram: Predicts context words from a target word
CBOW: Predicts a target word from context words
- Word2Vec captures semantic and syntactic relationships, making words with similar meanings appear close together in vector space.


4. **GloVe (Global Vectors):**

Uses global word co-occurrence statistics from large corpora. This approach:




Input: https://f1miamigp.com/news/press-release/beginners-guide-formula-1/

# **PyTorch**

Reference: https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html#an-example-n-gram-language-modeling

### **N-Gram Language Modeling**

An N-gram language model predicts a word given the previous N - 1 words. For example, if you're building a 3-gram model (trigram), it looks at the last two words in the sentence to predict the next word.

For example:

Text: "Amy played tennis at the country club today."
If the model sees "Amy played", it might predict "tennis" as the next word because it has learned from training data that this is a common sequence of words.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Define hyperparameters - trigram model
CONTEXT_SIZE = 2  # 2 words to predict third word
EMBEDDING_DIM = 10

# Input text
text = """Formula 1 is a complex sport. The technical and sporting
regulations extend to over 200 pages and there is a supplementary financial set
of rules for the teams and a drivers sporting code.
Here is a brief guide to explain the third-highest watched global sport,
behind the Olympics and FIFA World Cup. The FIA Formula 1 World Championship
started in 1950. With a series of races held in different locations around
the globe, the aim back then is the same as today. The winner is the first to
cross the finish line and points are awarded based on top ten positions — the
one with the most at the end of the year is crowned World Champion. In 1950
there were seven championship rounds, but as the sport has grown, next year
will feature 24 races. Starting in March and ending in November, the
championship competes across the world with races in Europe, Asia,
the Americas, the Middle East and Australia on a mixture of permanent,
street or hybrid tracks. Four of those venues were on the original schedule
73 years ago: Silverstone, Monaco, Spa and Monza.""".split()

# Build a vocabulary
vocab = set(text)
word_to_ix = {word: i for i, word in enumerate(vocab)}

# Create training data: ([context words], target word)
# For example: (['drivers', 'sporting'], 'code')
ngrams = [
    ([text[i - j - 1] for j in range(CONTEXT_SIZE)], text[i])
    for i in range(CONTEXT_SIZE, len(text))
]

# Print first 5 ngrams
print(ngrams[:5])

# Define the model
class NGramLanguageModeler(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

# Initialize the model
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Train the model
losses = []
for epoch in range(10):
    total_loss = 0
    for context, target in ngrams:
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Zero the gradients
        model.zero_grad()

        # Forward pass
        log_probs = model(context_idxs)

        # Compute loss
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    losses.append(total_loss)
print(losses)

# Get the embedding for a specific word (e.g., 'FIA')
print(model.embeddings.weight[word_to_ix['FIA']])

[(['1', 'Formula'], 'is'), (['is', '1'], 'a'), (['a', 'is'], 'complex'), (['complex', 'a'], 'sport.'), (['sport.', 'complex'], 'The')]
[906.0661115646362, 898.8450698852539, 891.7459230422974, 884.7731385231018, 877.932745218277, 871.2289490699768, 864.6665079593658, 858.2545833587646, 852.00590467453, 845.9321556091309]
tensor([-0.2614, -0.3543,  1.5285,  0.2608,  0.4529,  1.2677,  0.4860, -0.0886,
        -2.2568, -1.0339], grad_fn=<SelectBackward0>)


**Approach:**

* Each word in the vocabulary is initialized with a randomly generated dense vector (embedding).
* To predict the next word, the embedding vector of a context word is passed through two linear layers, with a ReLU activation function applied in between. This process transforms the embedding into a score vector, where each element represents a score for every word in the vocabulary.
* The score vector is then fed into a softmax function, which converts the raw scores (logits) into probabilities that sum to 1. The word with the highest probability is predicted as the next word in the sequence.

During training, the model adjusts the embeddings and weights of the linear layers to improve its ability to predict the next word across different contexts.






### **Computing Word Embeddings: Continuous Bag-of-Words**

A Continuous Bag-of-Words (CBOW) model predicts a word based on the surrounding context words. Instead of focusing only on the previous words, CBOW considers a few words both before and after the target word to make a prediction.

For example:
Text: "Amy played tennis at the country club today." If the model sees the context words "played" and "at", it might predict "tennis" as the target word because it has learned from the data that these words commonly appear around it.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

# Define hyperparameters - CBOW model
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
EMBEDDING_DIM = 10

# Input text (same as N-grams example)
text = """Formula 1 is a complex sport. The technical and sporting
regulations extend to over 200 pages and there is a supplementary financial set
of rules for the teams and a drivers sporting code.
Here is a brief guide to explain the third-highest watched global sport,
behind the Olympics and FIFA World Cup. The FIA Formula 1 World Championship
started in 1950. With a series of races held in different locations around
the globe, the aim back then is the same as today. The winner is the first to
cross the finish line and points are awarded based on top ten positions — the
one with the most at the end of the year is crowned World Champion. In 1950
there were seven championship rounds, but as the sport has grown, next year
will feature 24 races. Starting in March and ending in November, the
championship competes across the world with races in Europe, Asia,
the Americas, the Middle East and Australia on a mixture of permanent,
street or hybrid tracks. Four of those venues were on the original schedule
73 years ago: Silverstone, Monaco, Spa and Monza.""".split()

# Build a vocabulary
vocab = set(text)
word_to_ix = {word: i for i, word in enumerate(vocab)}

# Create training data: ([context words], target word)
# For example: (['is', 'Formula', 'a', 'complex'], '1')
data = [
    ([text[i - j - 1] for j in range(CONTEXT_SIZE)] +
     [text[i + j + 1] for j in range(CONTEXT_SIZE)], text[i])
    for i in range(CONTEXT_SIZE, len(text) - CONTEXT_SIZE)
]

# Print first 5 context-target pairs
print(data[:5])

# Define the model
class CBOW(nn.Module):
    def __init__(self, vocab_size, embedding_dim, context_size):
        super(CBOW, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * 2 * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)

    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))  # Flatten the embedding
        out = F.relu(self.linear1(embeds))  # Pass through first linear layer with ReLU
        out = self.linear2(out)  # Pass through second linear layer
        log_probs = F.log_softmax(out, dim=1)  # Apply log softmax to get probabilities
        return log_probs

# Initialize the model
model = CBOW(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Train the model
losses = []
for epoch in range(10):
    total_loss = 0
    for context, target in data:
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)

        # Zero the gradients
        model.zero_grad()

        # Forward pass
        log_probs = model(context_idxs)

        # Compute loss
        loss = loss_function(log_probs, torch.tensor([word_to_ix[target]], dtype=torch.long))

        # Backward pass and optimization
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    losses.append(total_loss)
print(losses)

# Get the embedding for a specific word (e.g., 'FIA')
print(model.embeddings.weight[word_to_ix['FIA']])

[(['1', 'Formula', 'a', 'complex'], 'is'), (['is', '1', 'complex', 'sport.'], 'a'), (['a', 'is', 'sport.', 'The'], 'complex'), (['complex', 'a', 'The', 'technical'], 'sport.'), (['sport.', 'complex', 'technical', 'and'], 'The')]
[895.7418150901794, 889.0682983398438, 882.4767565727234, 875.9580874443054, 869.5068650245667, 863.1163516044617, 856.7876577377319, 850.5198860168457, 844.316901922226, 838.1794955730438]
tensor([ 0.1546,  0.9563,  0.4431, -0.6779, -0.2627, -0.6793, -0.9609, -1.1815,
        -0.6050,  0.8514], grad_fn=<SelectBackward0>)


**Approach:**

* Each word in the vocabulary is initialized with a randomly generated dense vector (embedding).
* To predict the target word, the embedding vectors of the context words (a fixed number of words before and after the target) are combined by averaging them. This combined embedding represents the context. The combined context embedding is passed through two linear layers, with a ReLU activation function applied in between. This process transforms the combined context embedding into a score vector, where each element represents a score for every word in the vocabulary.
* The score vector is then passed through a softmax function, which converts the raw scores (logits) into probabilities. The word with the highest probability is predicted as the target word.

During training, the model adjusts both the embeddings of the words and the weights of the linear layers to improve its ability to predict the target word given the context across different sentences.

# **FastText**

Reference: https://radimrehurek.com/gensim/models/fasttext.html

* FastText breaks down each word into smaller pieces called character n-grams (short sequences of characters). For example, the word "playing" might be broken into parts like "pla", "lay", "ayi", "ing". FastText then learns vector representations for these subwords. So, the word "playing" is represented not only by its own vector but also by the combined vectors of these smaller subword pieces.

* When the model encounters a new word it hasn’t seen before, it can still generate an embedding by looking at the subwords. Example: The model has never seen the word "unplayable". Even though "unplayable" is new, you’ve seen its parts ("un", "play", "able") in other words. FastText uses the vectors of these subwords to form an approximate vector for "unplayable", which gives the model an idea of its meaning based on similar words.

* FastText’s use of subwords allows it to understand the morphological structure of words. Example: if the model has been trained on the word "run", it can still recognize and generate vectors for "running" or "runner" because it’s familiar with the root "run" and the suffixes "-ing" or "-ner".

In [ ]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 75.7 MB/s eta 0:00:00


In [ ]:
from gensim.models import FastText
from gensim.utils import tokenize

# Input text
text = """Formula 1 is a complex sport. The technical and sporting
regulations extend to over 200 pages and there is a supplementary financial set
of rules for the teams and a drivers sporting code.
Here is a brief guide to explain the third-highest watched global sport,
behind the Olympics and FIFA World Cup. The FIA Formula 1 World Championship
started in 1950. With a series of races held in different locations around
the globe, the aim back then is the same as today. The winner is the first to
cross the finish line and points are awarded based on top ten positions — the
one with the most at the end of the year is crowned World Champion. In 1950
there were seven championship rounds, but as the sport has grown, next year
will feature 24 races. Starting in March and ending in November, the
championship competes across the world with races in Europe, Asia,
the Americas, the Middle East and Australia on a mixture of permanent,
street or hybrid tracks. Four of those venues were on the original schedule
73 years ago: Silverstone, Monaco, Spa and Monza."""

# Tokenize the text
sentences = [list(tokenize(sentence)) for sentence in text.split(".")]

#sg ({0, 1}, optional) – Training algorithm: 1 for skip-gram; otherwise CBOW

# Skip-gram (sg=1)
skipgram_model = FastText(sentences=sentences, vector_size=10, window=2, min_count=1, sg=1, epochs=10)

# Get vector embedding for a word (e.g., 'Formula')
word_embedding_skipgram = skipgram_model.wv['Formula']
print("Word embedding for 'Formula' using Skip-gram:", word_embedding_skipgram)

# Most similar words to 'Formula'
similar_words_skipgram = skipgram_model.wv.most_similar('Formula')
print("Most similar words to 'Formula' using Skip-gram:", similar_words_skipgram)

# CBOW (sg=0)
cbow_model = FastText(sentences=sentences, vector_size=10, window=2, min_count=1, sg=0, epochs=10)

# Get vector embedding for a word (e.g., 'Formula')
word_embedding_cbow = cbow_model.wv['Formula']
print("Word embedding for 'Formula' using CBOW:", word_embedding_cbow)

# Most similar words to 'Formula'
similar_words_cbow = cbow_model.wv.most_similar('Formula')
print("Most similar words to 'Formula' using CBOW:", similar_words_cbow)


Word embedding for 'Formula' using Skip-gram: [-0.00037345 -0.01945035 -0.01565223 -0.01196962  0.00931256 -0.00693733
 -0.00285145  0.0151653  -0.01367982  0.01079412]
Most similar words to 'Formula' using Skip-gram: [('Cup', 0.8583016991615295), ('supplementary', 0.8039862513542175), ('complex', 0.7172372937202454), ('there', 0.7033318281173706), ('venues', 0.6953373551368713), ('grown', 0.6602954864501953), ('rounds', 0.6402096152305603), ('pages', 0.5786170363426208), ('technical', 0.5385828614234924), ('were', 0.5355334877967834)]
Word embedding for 'Formula' using CBOW: [-0.0004753  -0.01944755 -0.01558854 -0.01200746  0.00931861 -0.00678533
 -0.00287868  0.01509689 -0.0133948   0.01091775]
Most similar words to 'Formula' using CBOW: [('Cup', 0.8572816848754883), ('supplementary', 0.7973544597625732), ('complex', 0.7125447392463684), ('venues', 0.6898975372314453), ('there', 0.6826182007789612), ('grown', 0.6598688960075378), ('rounds', 0.6319658756256104), ('pages', 0.5657681822

# **Word2Vec**

Reference: https://radimrehurek.com/gensim/models/word2vec.html

Word2Vec is a model that learns word embeddings by predicting a word based on its surrounding context (CBOW) or predicting surrounding words given a target word (Skip-gram). Unlike FastText, Word2Vec does not break words into subwords; instead, it treats each word as a whole unit and learns a vector representation for each individual word based on its occurrence and co-occurrence with other words in the training data.
* cannot handle words it has never seen during training.
* cannot understand morphological structure.

In [ ]:
from gensim.models import Word2Vec
from gensim.utils import tokenize

# Input text
text = """Formula 1 is a complex sport. The technical and sporting
regulations extend to over 200 pages and there is a supplementary financial set
of rules for the teams and a drivers sporting code.
Here is a brief guide to explain the third-highest watched global sport,
behind the Olympics and FIFA World Cup. The FIA Formula 1 World Championship
started in 1950. With a series of races held in different locations around
the globe, the aim back then is the same as today. The winner is the first to
cross the finish line and points are awarded based on top ten positions — the
one with the most at the end of the year is crowned World Champion. In 1950
there were seven championship rounds, but as the sport has grown, next year
will feature 24 races. Starting in March and ending in November, the
championship competes across the world with races in Europe, Asia,
the Americas, the Middle East and Australia on a mixture of permanent,
street or hybrid tracks. Four of those venues were on the original schedule
73 years ago: Silverstone, Monaco, Spa and Monza."""

# Tokenize the text
sentences = [list(tokenize(sentence)) for sentence in text.split(".")]

# Skip-gram (sg=1)
skipgram_model_w2v = Word2Vec(sentences=sentences, vector_size=10, window=2, min_count=1, sg=1, epochs=10)

# Get vector embedding for a word (e.g., 'Formula')
word_embedding_skipgram_w2v = skipgram_model_w2v.wv['Formula']
print("Word embedding for 'Formula' using Skip-gram (Word2Vec):", word_embedding_skipgram_w2v)

# Most similar words to 'Formula'
similar_words_skipgram_w2v = skipgram_model_w2v.wv.most_similar('Formula')
print("Most similar words to 'Formula' using Skip-gram (Word2Vec):", similar_words_skipgram_w2v)

# CBOW (sg=0)
cbow_model_w2v = Word2Vec(sentences=sentences, vector_size=10, window=2, min_count=1, sg=0, epochs=10)

# Get vector embedding for a word (e.g., 'Formula')
word_embedding_cbow_w2v = cbow_model_w2v.wv['Formula']
print("Word embedding for 'Formula' using CBOW (Word2Vec):", word_embedding_cbow_w2v)

# Most similar words to 'Formula'
similar_words_cbow_w2v = cbow_model_w2v.wv.most_similar('Formula')
print("Most similar words to 'Formula' using CBOW (Word2Vec):", similar_words_cbow_w2v)

Word embedding for 'Formula' using Skip-gram (Word2Vec): [ 0.01138929 -0.01572289  0.02346409 -0.07858156 -0.0272467   0.02591916
  0.05410448 -0.02307482 -0.09582553  0.04407915]
Most similar words to 'Formula' using Skip-gram (Word2Vec): [('or', 0.66107177734375), ('technical', 0.6563730239868164), ('tracks', 0.6191846132278442), ('schedule', 0.5888702869415283), ('line', 0.559052050113678), ('awarded', 0.5013142228126526), ('on', 0.4786873459815979), ('November', 0.4729972779750824), ('finish', 0.47032687067985535), ('Monaco', 0.4550740718841553)]
Word embedding for 'Formula' using CBOW (Word2Vec): [ 0.01115528 -0.0158502   0.02272957 -0.07855552 -0.02740997  0.02644765
  0.05377872 -0.02383587 -0.0955544   0.04451155]
Most similar words to 'Formula' using CBOW (Word2Vec): [('or', 0.6617471575737), ('technical', 0.6541956663131714), ('tracks', 0.615896999835968), ('schedule', 0.5852609872817993), ('line', 0.5554044842720032), ('awarded', 0.5006256103515625), ('November', 0.471717327

# **GloVe**

Reference: https://www.geeksforgeeks.org/pre-trained-word-embedding-using-glove-in-nlp-models/

In [ ]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip

--2025-12-09 22:01:15--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-12-09 22:01:15--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-12-09 22:01:15--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load GloVe embeddings
def load_glove_embeddings(filepath, embedding_dim=50):
    embeddings_index = {}
    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            embeddings_index[word] = np.array(vector, dtype=np.float32)
    return embeddings_index

# Input text
text = """Formula 1 is a complex sport. The technical and sporting
regulations extend to over 200 pages and there is a supplementary financial set
of rules for the teams and a drivers sporting code.
Here is a brief guide to explain the third-highest watched global sport,
behind the Olympics and FIFA World Cup. The FIA Formula 1 World Championship
started in 1950. With a series of races held in different locations around
the globe, the aim back then is the same as today. The winner is the first to
cross the finish line and points are awarded based on top ten positions — the
one with the most at the end of the year is crowned World Champion. In 1950
there were seven championship rounds, but as the sport has grown, next year
will feature 24 races. Starting in March and ending in November, the
championship competes across the world with races in Europe, Asia,
the Americas, the Middle East and Australia on a mixture of permanent,
street or hybrid tracks. Four of those venues were on the original schedule
73 years ago: Silverstone, Monaco, Spa and Monza."""

# Tokenizer setup
tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])

# Convert text to sequences (list of word indices)
sequences = tokenizer.texts_to_sequences([text])
word_index = tokenizer.word_index

# Pad sequences to ensure uniform length
padded_sequences = pad_sequences(sequences, padding='post')

print("Word Index: ", word_index)
print("Padded Sequences: ", padded_sequences)

# Load GloVe embeddings (50-dimensional vectors)
glove_embeddings = load_glove_embeddings('glove.6B.50d.txt', embedding_dim=50)

# Get the embedding for a word (e.g., 'Formula')
def get_glove_embedding(word, embeddings):
    return embeddings.get(word, np.zeros(50))

# Example word embedding lookup for 'Formula'
word_embedding_glove = get_glove_embedding('Formula', glove_embeddings)
print("Word embedding for 'Formula' using GloVe:", word_embedding_glove)

# Create an embedding matrix for the tokenizer's vocabulary
def create_embedding_matrix(word_index, embeddings, embedding_dim=50):
    vocab_size = len(word_index) + 1
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    for word, i in word_index.items():
        embedding_vector = embeddings.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

# Create the embedding matrix using GloVe vectors
embedding_matrix = create_embedding_matrix(word_index, glove_embeddings, embedding_dim=50)

print("Embedding Matrix Shape: ", embedding_matrix.shape)

# Example: Get the dense vector for the first word in the vocabulary
first_word = list(word_index.keys())[0]
first_word_embedding = embedding_matrix[word_index[first_word]]
print(f"Dense vector for the word '{first_word}':", first_word_embedding)

Word Index:  {'the': 1, 'and': 2, 'is': 3, 'a': 4, 'in': 5, 'of': 6, 'world': 7, 'sport': 8, 'to': 9, 'championship': 10, 'with': 11, 'races': 12, 'on': 13, 'formula': 14, '1': 15, 'sporting': 16, 'there': 17, '1950': 18, 'as': 19, 'year': 20, 'were': 21, 'complex': 22, 'technical': 23, 'regulations': 24, 'extend': 25, 'over': 26, '200': 27, 'pages': 28, 'supplementary': 29, 'financial': 30, 'set': 31, 'rules': 32, 'for': 33, 'teams': 34, 'drivers': 35, 'code': 36, 'here': 37, 'brief': 38, 'guide': 39, 'explain': 40, 'third': 41, 'highest': 42, 'watched': 43, 'global': 44, 'behind': 45, 'olympics': 46, 'fifa': 47, 'cup': 48, 'fia': 49, 'started': 50, 'series': 51, 'held': 52, 'different': 53, 'locations': 54, 'around': 55, 'globe': 56, 'aim': 57, 'back': 58, 'then': 59, 'same': 60, 'today': 61, 'winner': 62, 'first': 63, 'cross': 64, 'finish': 65, 'line': 66, 'points': 67, 'are': 68, 'awarded': 69, 'based': 70, 'top': 71, 'ten': 72, 'positions': 73, '—': 74, 'one': 75, 'most': 76, 'at'

* Word Index: This dictionary maps words to their unique indices
* Paddded Sequences: These are sequences of word indices from the Word Index. Each number corresponds to a word in the original sentence. Padding ensures that all sequences have the same length, so shorter sentences are padded with zeros or additional tokens to match the length of the longest sentence.
* The output shows a zero vector, which indicates that the word 'formula' was not found in the GloVe pre-trained embeddings.
* The shape of the embedding matrix (120, 50) indicates that there are 120 words (or unique indices) in your vocabulary, and each word is represented by a 50-dimensional vector.